# examples for `hostess.aws.ec2`

### introduction

`hostess.aws.ec2` is a collection of utilities for working with EC2 instances.

`Instance` and `Cluster` are its centerpiece classes. They are abstractions for, 
respectively, single EC2 instances and groups of EC2 instances. By offering both
managed interaction with the EC2 API and a rich set of remote procedure call (RPC)
capabilities, they attempt to make distributed workflows as conceptually simple and 
immediate as local ones. 

### requirements

1.  You need appropriate AWS permissions to perform any actions that call the
    EC2 API. You cannot, for instance, use `ls_instances()` without the ListInstances
    permission, or `Instance.start()` without StartInstances permission for the
    particular instance you are attempting to start. (A complete discussion of AWS
    permissions management is beyond the scope of this document.)

    By default, `hostess` uses the 'default' profile from ~/.aws/credentials.
    This can be modified in `hostess.config.user_config` or by manually constructing
    a session with `hostess.aws.utils.init_session`. 

    If you are in a situation where you have SSH access to, but not AWS permissions for, 
    an EC2 instance you'd like to make `hostess` RPCs on, you should ignore the fact that
    it is an EC2 instance and simply use `hostess.ssh.SSH`, which underlies the RPC
    capabilities of this module.
    
3. The RPC functionality offered by this module relies on SSH, so you need SSH access to an
   instance to make RPCs on it. Specifically:
   
   1. The instance must be running `sshd` and configured to accept incoming connections.
      You generally will not have to do anything special to set this up: the default
      configuration of most stock AMIs, including the Ubuntu and Amazon Linux images, is
      suitable.
   2. The inbound traffic rules of the instance's security group must permit SSH access
      from your IP.
   3. `hostess` supports only keyfile-based authentication, so you must have the correct
      keyfile for the instance, and `hostess` must be able to find it. If it shares a
      filename with the key name known to AWS and is in your `~/.ssh` folder, `hostess`
      will find it automatically. If this is not the case, you can manually specify a
      path to the keyfile when constructing an `Instance` or `Cluster`. You can also change
      the default search paths in `hostess.config.user_config`.

   *note: if you want to create remote workflows without relying on SSH connections, we recommend
   looking at the `hostess.station` framework.* 

### relationship to `boto3`

`aws.ec2.Instance` is designed to be easier to use, more Pythonic, and more powerful than `boto3`'s 
`Instance` abstraction. However, it does not implement high-level interfaces for the entirety 
of the EC2 API. Because it works partly by wrapping parts of `boto3`, however, every `aws.ec2.Instance` 
also grants access to a `boto3` `Instance` object with a shared AWS configuration. If 
you need access to other parts of the EC2 API inside a `hostess` workflow, you can access this object
via the `aws.ec2.Instance._instance` attribute.

## 1. listing and finding instances

*note: the output of these examples won't be very interesting if you don't currently
have any instances. if you don't have any, but you do have an AWS profile configured on 
your computer, you might want to skip down to the 'launching instances' section.*

If you have ever used the `awscli` command `aws ec2 describe-instances` to try to find
an EC2 instance, you may have noticed that although its output is very complete, it is
extremely verbose, deeply nested, and hard to parse. Its filtering options are also 
somewhat difficult to use. `hostess`'s `ls_instances()` is a much more lightweight 
alternative that is equally suitable for most use cases.

In [ ]:
# if you simply call `ls_instances()` with no arguments, it will return a tuple
# of dicts giving essential information about all your running, pending, or stopped
# instances:
# name (if any); public ip (if any); instance id; state (running, 
# pending, stopped, terminated); private ip (if any), and keyname (if any). 
from hostess.aws.ec2 import ls_instances

instance_info = ls_instances()
instance_info[0:2]

In [ ]:
# ls_instances() offers a variety of ways to filter your search.
# see the docstring for a full set of options; we'll just describe
# one here. ls_instances() understands arbitrary keywords arguments 
# as case-insensitive tag filters. 
# if you have a single instance named 'kitty', you can find it with:
ls_instances(name='kitty')[0]

In [ ]:
# this feature also supports optional regex matching. if you have a 
# set of instances named 'pipeline_1', 'pipeline_2', etc.:
ls_instances(name=r'pipeline_\d', tag_regex=True)

## 2. connecting to an existing instance

The only required argument to the `Instance` constructor is an
identifier for the EC2 instance you'd like to work with. There are three
acceptable types of identifier:

* a connectable IP address for the instance, like `"102.31.4.129"`
* the AWS instance identifier, like `"i-0868ad3eeebe16cde"`
* one of the `dicts` returned by `ls_instances()`

*note: if you're connecting to an instance from another EC2 instance, 
pass `use_private_ip=True` to the `Instance` constructor, and if 
you're using an IP address as the identifier in this case, make sure
it's the private IP.*

In [ ]:
# so, returning to the instance uniquely named 'kitty', 
# let's construct an Instance object connected to it 
# using a dict from ls_instances(): 
from hostess.aws.ec2 import Instance, ls_instances

kitty = Instance(ls_instances(name='kitty')[0])

# the string representation of an Instance shows you its name (if any),
# its instance id, its instance type, its EC2 zone, and the IP address
# you're using to connect to it (if it's running). it also has a number 
# of 'basic information' attributes. here are a few:
print(kitty)
print(
    f"the instance is {kitty.state}, named '{kitty.name}', "
    f"and has the following tags: {kitty.tags}"
)

In [ ]:
# you generally don't have to manually establish an SSH connection to the instance --
# it will happen automatically when you try to use functionality that
# requires a connection. However, if your connection is externally disrupted due to network
# dropouts or other issues, you might need to manually reestablish it. This method can 
# also be used to ensure that you _can_ connect to the instance, or to 'prep' it so that 
# there's no connection delay on your first command when you get to that part of a program:
kitty.connect()

## 3. controlling instance state

`Instance` has several methods to control an instance's activation state:

* `start()` boots the Instance.
* `stop()` shuts the instance down.
* `restart()` shuts the instance down, waits for complete shutdown, and boots it again.
* `terminate()` terminates the instance. 
  * **\*\*IMPORTANT\*\***: this permanently and irrevocably deletes an instance and, unless
    it's specifically configured otherwise, its root volume.
  * `hostess` trusts that you know what you're doing, so `terminate()`doesn't have any special guardrails.
  * `Instance.terminate()` is like `sudo rm -rf`: Don't even type it unless you really mean it!

These methods don't do anything if the instance is already in (or transitioning to)
the requested state.

In [ ]:
# if you're done with kitty for the moment and don't want to incur further on-demand charges:
kitty.stop()

In [ ]:
# and if you change your mind:
kitty.start()

In [ ]:
# we're not going to type out the terminate command.

## 4. remote procedure calls

`Instance` supports two main types of remote procedure calls (RPCs):

* shell commands
* Python function calls

### 4a. shell commands

With `Instance`, you can run commands as if you were logged 
into the instance and work with the output of those commands in Python. 
`Instance` has three primary methods for this. They are all highly 
configurable, and we don't discuss all their options here. See the 
documentation for `hostess.subutils.RunCommand` and 
`hostess.subutils.Viewer` for a full description of options.

* `command()` runs a command in the remote user's default login shell.
  you can pass a command as a literal string, or construct it from
  Python arguments (examples below). By default it runs the command
  asynchronously and returns a `hostess.subutils.Viewer` object you
  can use to inspect or terminate the process. 
* `con()` simulates the experience of typing a command into a console
  and looking at its output. It blocks until the process exits and
  pretty-prints any output from the process.
* `commands()` provides syntactic sugar for constructing list commands.
  
*note: `hostess` only fully supports `bash`. some functionality may not work in other shells.*

In [ ]:
# get the full UNIX name information for the instance.
# `hostess` interprets the a=True argument as a shell 
# switch; this is equivalent to kitty.command('uname -a').
uname = kitty.command('uname', a=True)

# because Instance.command() runs asynchronously by default,
# it's unlikely that you'll get any output in the microseconds
# it takes to get to this next line. you'll probably just see 
# that 'uname' is a Viewer for a running 'uname -a' process. 
uname

In [ ]:
# it'll probably have completed by the time you execute
# this cell, though. 
uname

In [ ]:
# if you'd like to ensure that a command completes before you move on 
# to the next part of your code, you can call the .wait() method of 
# the returned Viewer object or pass _wait=True:
usage = kitty.command("df -h", _wait=True)
usage

In [ ]:
# you can access the stdout and stderr of commands you execute
# via the .out and .err attributes of returned Viewers.

# these are lists of strings. each element of the lists is an 
# individual write to stdout/stderr by the remote process. 
# simple commands that do a thing and exit will generally only have one element, 
# because they return their output all at once. (this may not be
# true in cases in which the output is extremely large, due to SSH 
# buffering, etc.
print(f"{len(usage.out)} write(s) to stdout")

In [ ]:
# this allows you to use the results of remote shell commands in code.
import re
import pandas as pd

rows = [
    re.split(' +', line, maxsplit=5) 
    for line in usage.out[0].splitlines()
]
pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
# it also allows you to monitor the results of ongoing processes.
# a silly example:
kitty.command("echo 1 > numbers.txt", _wait=True)
tail = kitty.command("tail -f numbers.txt")
number = 1
while len(tail.out) < 5:
    number += 1
    kitty.command(f"echo {number} >> numbers.txt", _wait=True)
    print(tail.out)

In [ ]:
# it can sometimes be important to do manual cleanup of backgrounded RPCs. 
# note that the tail process is still running, even though we're not using it anymore:
tail.running

In [ ]:
# so you might want to kill it.
tail.kill()
tail.running

In [ ]:
# the Instance.con() method blocks until process exit, pretty-prints its 
# output, and doesn't return anything. It's intended to give the feel of
# running a command in a console. to pretty-print all active TCP connections:
kitty.con("ss -t")

In [ ]:
# Instance.commands() is an easy way to perform a sequence of commands
# without having to enter a monolithic string. a silly example:
kitty.commands(["cd /", "ls"], _wait=True)

In [ ]:
# this can be used for serious sysadmin stuff. 
# a real-world example might look like this:
from itertools import chain

private_repos = ["sensitive_devops", "proprietary_algos", "company_secrets"]
update_commands = chain(
    *[
        (f"ssh-add .ssh/{repo}_deploy", f"cd {repo}", "git pull", "cd ~")
        for repo in private_repos
    ]
)
update_result = kitty.commands(
    ["eval `ssh-agent`", *update_commands], op="and", _wait=True
)

# this won't actually work, of course, because these are 
# hypothetical keys and repos, but you get the idea.
print(update_result.err)

# note that op="and" caused hostess to chain the long sequence of commands
# with &&, making bash terminate the whole sequence if any one failed:
update_result.command